# SO-ARM 机械臂安装和环境设置

本教程基于 Seeed Studio 官方指南，完整介绍 SO-ARM100/SO-ARM101 机械臂的环境搭建和初始设置。

**适用硬件：**
- SO-ARM100 (Standard 双臂套装)
- SO-ARM101 (Pro 版本)

**系统要求：**
- Ubuntu 22.04 (X86) + CUDA 12+ + Python 3.10
- 或 Jetson Orin (Jetpack 6.0/6.1) + Python 3.10

**参考文档：** [Seeed Studio LeRobot Wiki](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/)

---
## 📋 步骤 0: 硬件检查清单

### SO-ARM100 标准版

**Leader Arm (主动臂)：**
- 7 个 ST-3215-C001 舵机 (7.4V, 减速比 1:345)
- 5V 4A 电源适配器
- USB 转串口线

**Follower Arm (从动臂)：**
- 7 个 ST-3215-C001 舵机 (7.4V, 减速比 1:345)
- 5V 4A 电源适配器
- USB 转串口线

### SO-ARM101 Pro 版

**Leader Arm：**
- 混合减速比舵机 (C044/C001/C046)
- 5V 4A 或 12V 电源（根据版本）

**Follower Arm：**
- 与 SO-ARM100 相同配置

### 可选配件
- USB 摄像头（推荐 640x480 @ 30fps）
- 夹爪末端执行器

**重要提示：** 确保所有硬件已正确组装并通电测试！

---
## 🔧 步骤 1: 创建 Conda 环境

In [ ]:
# 注意：这些命令需要在终端中运行，不是在 Notebook 中
# 这里仅作为参考和记录

print("请在终端运行以下命令：")
print()
print("# 1. 创建并激活 conda 环境")
print("conda create -y -n lerobot python=3.10")
print("conda activate lerobot")
print()
print("# 2. 安装 ffmpeg")
print("conda install ffmpeg=7.1.1 -c conda-forge")
print()
print("注意：如果遇到视频流问题，必须使用 ffmpeg 7.1.1 版本！")

---
## 📦 步骤 2: 克隆并安装 LeRobot

**推荐使用 Seeed Projects 维护的分支，包含最新的 SO-ARM 支持。**

In [ ]:
print("请在终端运行以下命令：")
print()
print("# 1. 克隆 Seeed Projects 的 LeRobot 仓库")
print("git clone https://github.com/Seeed-Projects/lerobot.git ~/lerobot")
print()
print("# 2. 进入目录")
print("cd ~/lerobot")
print()
print("# 3. 安装 LeRobot 及 Feetech 舵机支持")
print("pip install -e '.[feetech]'")
print()
print("# 这将安装：")
print("# - LeRobot 核心库")
print("# - Feetech 舵机驱动")
print("# - 所有必需的依赖")

---
## ✅ 步骤 3: 验证安装

In [ ]:
# 检查 LeRobot 是否正确安装
import sys

print("检查 Python 环境...")
print(f"Python 版本: {sys.version}")
print(f"Python 路径: {sys.executable}")
print()

# 检查 LeRobot
try:
    import lerobot
    print(f"✓ LeRobot 已安装: {lerobot.__version__}")
except ImportError as e:
    print(f"✗ LeRobot 未安装: {e}")
    print("请运行: pip install -e '.[feetech]'")

# 检查 Feetech 支持
try:
    from lerobot.robots.so101_follower import SO101Follower
    print("✓ SO101 Follower 支持已安装")
except ImportError:
    print("✗ SO101 支持缺失")

try:
    from lerobot.teleoperators.so101_leader import SO101Leader
    print("✓ SO101 Leader 支持已安装")
except ImportError:
    print("✗ SO101 Leader 支持缺失")

# 检查 PyTorch
try:
    import torch
    print(f"✓ PyTorch 已安装: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"✓ CUDA 可用: {torch.cuda.get_device_name(0)}")
    else:
        print("⚠️ CUDA 不可用 (训练将很慢)")
except ImportError:
    print("✗ PyTorch 未安装")

print()
print("安装检查完成！")

---
## 🔌 步骤 4: 连接和识别机械臂

### 4.1 物理连接

1. **连接 Leader Arm（主动臂）**
   - 插入 5V 4A 电源（或 12V for Pro 版）
   - 连接 USB 转串口线到电脑
   - 确保电源指示灯亮起

2. **连接 Follower Arm（从动臂）**
   - 插入 5V 4A 电源
   - 连接 USB 转串口线到电脑（使用不同的 USB 口）
   - 确保电源指示灯亮起

### 4.2 识别串口设备

In [ ]:
import subprocess

print("检测可用的串口设备...")
print()

result = subprocess.run(['ls', '-l', '/dev/ttyACM*'], 
                       capture_output=True, text=True)

if result.returncode == 0:
    print("找到以下串口设备：")
    print(result.stdout)
    print()
    print("通常：")
    print("- /dev/ttyACM0 → Follower Arm (从动臂)")
    print("- /dev/ttyACM1 → Leader Arm (主动臂)")
    print()
    print("如果顺序不同，请在后续步骤中调整端口号")
else:
    print("❌ 未检测到串口设备！")
    print()
    print("排查步骤：")
    print("1. 检查 USB 线是否插好")
    print("2. 检查电源是否接通")
    print("3. 尝试重新插拔 USB 线")
    print("4. 运行: ls -l /dev/ttyUSB* (有些系统使用 ttyUSB)")

### 4.3 设置串口权限

In [ ]:
print("如果遇到权限错误，请在终端运行：")
print()
print("sudo chmod 666 /dev/ttyACM*")
print()
print("或者将用户添加到 dialout 组（永久解决）：")
print("sudo usermod -a -G dialout $USER")
print("# 然后注销并重新登录")

---
## 🎯 步骤 5: 配置舵机 ID

**重要：** 在首次使用机械臂前，必须为每个舵机分配正确的 ID。

### 配置 Follower Arm

In [ ]:
print("在终端运行以下命令配置 Follower Arm：")
print()
print("lerobot-setup-motors \\")
print("  --robot.type=so101_follower \\")
print("  --robot.port=/dev/ttyACM0")
print()
print("按照提示操作：")
print("1. 确保只有一个舵机连接到总线")
print("2. 程序会扫描并找到舵机")
print("3. 为舵机分配 ID (从 1 开始)")
print("4. 重复此过程直到所有 7 个舵机都配置完成")
print()
print("⚠️ 注意事项：")
print("- 每次只连接一个舵机")
print("- 按照从底座到末端的顺序配置")
print("- 记录每个舵机的 ID 和位置")

### 配置 Leader Arm

In [ ]:
print("在终端运行以下命令配置 Leader Arm：")
print()
print("lerobot-setup-motors \\")
print("  --teleop.type=so101_leader \\")
print("  --teleop.port=/dev/ttyACM1")
print()
print("重复与 Follower Arm 相同的配置过程")

---
## 🔧 常见问题和解决方案

### 问题 1: "Motor not found"

**原因：** 舵机未通电或连接线松动

**解决：**
1. 检查电源是否接通（指示灯是否亮起）
2. 检查通信线是否插好
3. 确认电压正确（7.4V 舵机用 5V 电源）
4. 尝试重新插拔连接线

### 问题 2: "Could not connect on port /dev/ttyACM0"

**原因：** 权限不足

**解决：**
```bash
sudo chmod 666 /dev/ttyACM*
```

### 问题 3: "No valid stream found"

**原因：** ffmpeg 版本不兼容

**解决：**
```bash
conda install ffmpeg=7.1.1 -c conda-forge
```

### 问题 4: "Magnitude exceeds 2047"

**原因：** 舵机位置超出范围或未校准

**解决：**
1. 断电重启机械臂
2. 重新运行校准程序
3. 检查机械臂是否在正确的初始位置

### 问题 5: 舵机ID冲突

**原因：** 多个舵机有相同的 ID

**解决：**
1. 每次只连接一个舵机到总线
2. 使用 `lerobot-setup-motors` 重新分配 ID
3. 确保每个舵机有唯一的 ID (1-7)

---
## ✅ 安装完成检查清单

在继续下一步之前，请确认：

- [ ] Conda 环境已创建并激活
- [ ] LeRobot 已成功安装（带 feetech 支持）
- [ ] ffmpeg 7.1.1 已安装
- [ ] 两个机械臂都已连接并被识别为 /dev/ttyACM0 和 /dev/ttyACM1
- [ ] 串口权限已正确设置
- [ ] Follower Arm 的所有舵机 ID 已配置 (1-7)
- [ ] Leader Arm 的所有舵机 ID 已配置 (1-7)
- [ ] 没有硬件错误或警告信息

### 下一步

完成安装后，请继续：

**01_calibration_and_teleoperation.ipynb** - 校准机械臂并测试遥操作

---

**参考资源：**
- [Seeed Studio LeRobot Wiki](https://wiki.seeedstudio.com/cn/lerobot_so100m_new/)
- [LeRobot 官方文档](https://huggingface.co/docs/lerobot)
- [GitHub Issues](https://github.com/Seeed-Projects/lerobot/issues)